# 1 Extracting genes and variants

In [1]:
import pandas as pd
import math
from pathlib import Path
import numpy as np
import re
import glob

In [2]:
# incase you've ran the prev notebook on splits of papers
data = []
for file in glob.glob("data/model_output/*.csv"):
    print(file)
    # 'WBPaper ID', 'Method', '* Genes', '* Gene-Variant combo', 'Mutation', 'Sentence'
    text = pd.read_csv(file).to_numpy().tolist()
    data = data + text 
data = np.array(data)

data/model_output/extracted_snippets.csv
data/model_output/extracted_snippets_part2.csv
data/model_output/extracted_snippets_part3.csv


In [3]:
OPENING_CLOSING_REGEXES = [r'((?:^|[\s\(\[\'"/,;\-])', r'(?:^|[\s\(\[\'"/,;\-]))']

# the allele regex and db idea was stolen from wbtools
allele_designations = np.load('data/gsoc/wbtools/wb_allele_designations.npy').astype('U6')
alleles_variations = np.load('data/gsoc/wbtools/wb_alleles_variations.npy').astype('U6')
DB_VAR_REGEX = r'({designations}|m|p|ts|gf|lf|d|sd|am|cs)([0-9]+)'
var_regex_1 = OPENING_CLOSING_REGEXES[0] + DB_VAR_REGEX.format(designations="|".join(allele_designations)) + OPENING_CLOSING_REGEXES[1]
all_var = OPENING_CLOSING_REGEXES[0] + '|'.join(alleles_variations) + '|' + var_regex_1 + OPENING_CLOSING_REGEXES[1]
all_var = [re.compile(r,re.IGNORECASE) for r in [all_var]]

# 'WBPaper ID', 'Method', '* Genes', 'Variants', '*Gene-Variant combo ', 'Mutations', 'Sentence'
updated_data = []
total = len(data)
print('Total sentences: {}, processed count: '.format(total), end=' ')
for i, sent in enumerate(data[:, -1]):
    if (i+1) % 100 == 0: print(f"{i+1}", end = " ")
    variants = []
    for regex in all_var:      
        for m in regex.finditer(sent):
            span = (m.start(0), m.end(0))    
            raw = (sent[span[0]:span[1]]).strip()
            raw = raw[1:] if not raw[0].isalnum() else raw
            raw = raw[:-1] if not raw[-1].isalnum() else raw
            if len(raw.strip()) > 1: variants.append(raw.strip())
    if variants:
        variants  = list(set(variants))
        variants = "'" + "', '".join(variants) + "'"
    else:
        variants = ''
    updated_data.append([data[i,0], data[i,1], data[i,2], variants, data[i,-3], data[i,-2], data[i,-1]])

Total sentences: 19001, processed count:  100 200 300 400 500 600 700 800 900 1000 1100 1200 1300 1400 1500 1600 1700 1800 1900 2000 2100 2200 2300 2400 2500 2600 2700 2800 2900 3000 3100 3200 3300 3400 3500 3600 3700 3800 3900 4000 4100 4200 4300 4400 4500 4600 4700 4800 4900 5000 5100 5200 5300 5400 5500 5600 5700 5800 5900 6000 6100 6200 6300 6400 6500 6600 6700 6800 6900 7000 7100 7200 7300 7400 7500 7600 7700 7800 7900 8000 8100 8200 8300 8400 8500 8600 8700 8800 8900 9000 9100 9200 9300 9400 9500 9600 9700 9800 9900 10000 10100 10200 10300 10400 10500 10600 10700 10800 10900 11000 11100 11200 11300 11400 11500 11600 11700 11800 11900 12000 12100 12200 12300 12400 12500 12600 12700 12800 12900 13000 13100 13200 13300 13400 13500 13600 13700 13800 13900 14000 14100 14200 14300 14400 14500 14600 14700 14800 14900 15000 15100 15200 15300 15400 15500 15600 15700 15800 15900 16000 16100 16200 16300 16400 16500 16600 16700 16800 16900 17000 17100 17200 17300 17400 17500 17600 17700 1780

In [4]:
# above cell takes a while to complete, so saving the data temporarily
updated_data = pd.DataFrame(updated_data[:], columns=['WBPaper ID', 'Method', '*Genes', 'Variants', '*Gene-Variant combo ', 'Mutations', 'Sentence'])
updated_data.to_csv("data/model_output/processed/snippets_1.csv", index=False, encoding='utf-8')
updated_data = None

# 2 Normalizing genes to WB dictionary

In [5]:
data = pd.read_csv("data/model_output/processed/snippets_1.csv")
data = data.to_numpy() # 'WBPaper ID', 'Method', 'Genes', 'Variants', '*Gene-Variant combo ', 'Mutations', 'Sentence'

In [6]:
wb_genes_1 = Path('data/gsoc/Gene_alias.1.txt').read_text().split('\n')
wb_genes_2 = Path('data/gsoc/Gene_alias.2.txt').read_text().split('\n')
wb_genes_3 = Path('data/gsoc/Gene_alias.3.txt').read_text().split('\n')

wb_genes_1 = [r.split('\t') for r in wb_genes_1]
wb_genes_2 = [r.split(' ') for r in wb_genes_2]
wb_genes_3 = [r.split(' ') for r in wb_genes_3]

Inefficient way to do this. Have to work on better search algo.

In [7]:
all_wb_genes = dict()

for row in wb_genes_1+wb_genes_2+wb_genes_3:
    if row[0] not in all_wb_genes.keys():
        all_wb_genes[row[0]] = []
    for gene in row[1:]: 
        if len(gene) and gene.lower() not in all_wb_genes[row[0]]: 
            all_wb_genes[row[0]].append(gene.lower())
len(all_wb_genes)

306123

In [8]:
print('Total sentences: {}, processed count: '.format(len(data)), end=' ')
updated_data = []

for i, genes in enumerate(data[:, 2]):
    if (i+1) % 100 == 0: print(f"{i+1}", end = " ")
    # checking if nan
    if type(genes) == float:
        col_genes = ''
    else:
        genes = genes[1:-1].split("', '")
        col_genes = []
        
        for gene in genes:
            for key, value in all_wb_genes.items():
                if gene.lower() in value:
                    col_genes.append(key)
                    break
        if col_genes:
            col_genes = list(set(col_genes))
            col_genes = "'" + "', '".join(col_genes) + "'"
        else: 
            col_genes = ''
    updated_data.append([data[i,0], data[i,1], data[i,2], col_genes, data[i,3], data[i,4], data[i,5], data[i,6]])
    
data = updated_data # 'WBPaper ID', 'Method', 'Genes', 'WBGenes', 'Variants', '*Gene-Variant combo ', 'Mutations', 'Sentence'
updated_data = None

Total sentences: 19001, processed count:  100 200 300 400 500 600 700 800 900 1000 1100 1200 1300 1400 1500 1600 1700 1800 1900 2000 2100 2200 2300 2400 2500 2600 2700 2800 2900 3000 3100 3200 3300 3400 3500 3600 3700 3800 3900 4000 4100 4200 4300 4400 4500 4600 4700 4800 4900 5000 5100 5200 5300 5400 5500 5600 5700 5800 5900 6000 6100 6200 6300 6400 6500 6600 6700 6800 6900 7000 7100 7200 7300 7400 7500 7600 7700 7800 7900 8000 8100 8200 8300 8400 8500 8600 8700 8800 8900 9000 9100 9200 9300 9400 9500 9600 9700 9800 9900 10000 10100 10200 10300 10400 10500 10600 10700 10800 10900 11000 11100 11200 11300 11400 11500 11600 11700 11800 11900 12000 12100 12200 12300 12400 12500 12600 12700 12800 12900 13000 13100 13200 13300 13400 13500 13600 13700 13800 13900 14000 14100 14200 14300 14400 14500 14600 14700 14800 14900 15000 15100 15200 15300 15400 15500 15600 15700 15800 15900 16000 16100 16200 16300 16400 16500 16600 16700 16800 16900 17000 17100 17200 17300 17400 17500 17600 17700 1780

Checking if any detected gene was NOT in the WB gene dictionary

In [9]:
data = np.array(data)
data[len(data[:,2]) != len(data[:,3])] 

array([], shape=(0, 19001, 8), dtype='<U11773')

In [10]:
# above cell takes a while to complete, so saving the data temporarily
data = data.tolist()
data = pd.DataFrame(data[:], columns=['WBPaper ID', 'Method', 'Genes', 'WBGenes', 'Variants', '*Gene-Variant combo ', 'Mutations', 'Sentence'])
data.to_csv("data/model_output/processed/snippets_2.csv", index=False, encoding='utf-8')
data = None

# 3 Normalizing mutations to  one-letter amino acid codes

These code imports would be doing the same thing done in notebook 2, but on a much small subset of data.   
TODO later, not code breaking: This additional metadata of how the mutation was extracted should be inside notebook 2.

In [12]:
data = pd.read_csv("data/model_output/processed/snippets_2.csv")
data = data.to_numpy() # 'WBPaper ID', 'Method', 'Genes', 'WBGenes', 'Variants', '*Gene-Variant combo ', 'Mutations', 'Sentence'

In [13]:
import configparser

from utils.misc.regex_block import mutation_finder_from_regex_filepath, TmVar, CustomWBregex, normalize_mutations

In [14]:
db_config = configparser.ConfigParser()
db_config.read('utils/all_config.cfg')

custom_mut_extract = CustomWBregex(db_config, locus_only=True)
mf_mut_extract = mutation_finder_from_regex_filepath('data/regexs/mutationfinder_regex/seth_modified.txt')
tmvar_mut_extract = TmVar('data/regexs/tmvar_regex/final_regex_path')

/mnt/c/Users/Rishab/Documents/GitHub/genomic-info-from-papers/genomic-info-from-papers/utils/misc/regex_block.py:302: FutureWarning: Possible nested set at position 15
  self._regular_expressions.append(re.compile(reg))


In [15]:
def point_mut_block(sentence, span_size=150):
    mut_and_snippets = []
    
    # MutationFinder
    for mutation, snip in mf_mut_extract(raw_text=sentence, span_size=span_size).items():
        mut_and_snippets.append([mutation.OriginalMention, snip])
    # tmVar
    mut_and_snippets = mut_and_snippets + tmvar_mut_extract(sentence, span_size=span_size)
    # Custom patterns
    mut_and_snippets = mut_and_snippets + custom_mut_extract(sentence, span_size=span_size)

    if mut_and_snippets:
        mut_and_snippets = np.array(mut_and_snippets)
        mut_and_snippets = mut_and_snippets[:, 0].tolist()
        mut_and_snippets = list(set(mut_and_snippets))
    return mut_and_snippets

In [16]:
point_mut_block('ad465 , nucleotide 2862 of the coding region to 400 bp downstream G-to-A change at nucleotide 673 resulting in a stop codon from the stop site was amplified using primers ATGGATGAAC at amino acid 107; ad692 , T-to-G change at nucleotide 811 TATACAA')

['T-to-G change at nucleotide 811', 'G-to-A change at nucleotide 673']

In [17]:
normalize_mutations('T-to-G change at nucleotide 811'), normalize_mutations('Phe230Leu'), \
normalize_mutations('C to T at nucleotide 4539'), normalize_mutations('methionine for lysine-856'), \
normalize_mutations('glycine-118 is replaced by an arginine'), normalize_mutations('1247 (valine to leucine')

('T811G', 'F230L', 'C4539T', 'M856K', 'G118R', 'V1247L')

Working with the protein mutations from regex block for now

In [18]:
# old - 'WBPaper ID', 'Method', 'Genes', 'WBGenes', 'Variants', '*Gene-Variant combo ', 'Mutations', 'Sentence'
# new - 'WBPaper ID', 'Method', 'Genes', 'WBGenes', 'Variants', '*Gene-Variant combo ', 'Mutations', 'Normalized Mutations', 'Sentence'
temp = []
total_count = len(data)
ner_count = 0
regex_count = 0
print('Following mutations could NOT be normalized. Either a) normalize them manually and add in the csv file b) Make edits in the normalize_mutations fn')
for i, row in enumerate(data):
    if row[1] != 'Regex':
        if row[1] == 'NER':
            ner_count += 1
        temp.append(np.insert(data[i], -1, '').tolist())
    else:
        regex_count += 1
        norm_mutations = []
        mutations = data[i, -2][1:-1].split("', '")
        for raw_mut in mutations: 
            mut = point_mut_block(raw_mut)
            if mut:
                for m in mut:
                    m = m.replace(",", "")
                    try:
                        norm_mut = normalize_mutations(mut[0])
                        norm_mutations.append(norm_mut)
                    except KeyError:
                        print(m)
        if norm_mutations:
            norm_mutations = list(set(norm_mutations))
            norm_mutations = "'" + "', '".join(norm_mutations) + "'"
        else: 
            norm_mutations = ''
        temp.append(np.insert(data[i], -1, norm_mutations).tolist())
        
data = temp
temp = None

Following mutations could NOT be normalized. Either a) normalize them manually and add in the csv file b) Make edits in the normalize_mutations fn
53 bp del
serine at position 377 to phenylalanine
arginine at position 551 with a histidine
glycine at position 573 with a serine
aspartic acid to asparagine at codon 652
glutamine at codon 13 to a stop
glutamic acid 230 to lysine
glycine at position 560 to an arginine
glycine at position 558 to an arginine
glycine at position 76 changed to glutamic acid
glycine at position 76 changed to glutamic acid
glutamic acid for glycine at codon 13


In [19]:
print('All', ner_count, 'NER data row was ignored. Only', regex_count, 'regex data rows were used.')

All 1400 NER data row was ignored. Only 626 regex data rows were used.


In [20]:
# saving things
data = pd.DataFrame(data[:], columns=['WBPaper ID', 'Method', 'Genes', 'WBGenes', 'Variants', '*Gene-Variant combo ', 'Mutations', 'Normalized Mutations', 'Sentence'])
data.to_csv("data/model_output/processed/snippets_3.csv", index=False, encoding='utf-8')

# 4 Validation

In [21]:
data = pd.read_csv("data/model_output/processed/snippets_3.csv")
data = data.to_numpy() # 'WBPaper ID', 'Method', 'Genes', 'WBGenes', 'Variants', '*Gene-Variant combo ', 'Mutations', 'Normalized Mutations', 'Sentence'

In [22]:
proteinfa = Path('data/gsoc/proteinfa/c_elegans.PRJNA13758.WS281.protein.fa').read_text().split('>')[1:]

In [23]:
wb_gene_and_prot = dict() # {wbgene: [transcript, protein]}

for row in proteinfa:
    wbgene = re.findall("WBGene[0-9]+", row)[0]
    protein = "".join(re.findall("\n.*", row)).replace('\n','')
    transcript = row.split(' ')[0]
    if wbgene not in wb_gene_and_prot.keys():
        wb_gene_and_prot[wbgene] = []
    wb_gene_and_prot[wbgene].append([transcript, protein])
    
len(wb_gene_and_prot)

19987

In [24]:
# threshold of how close the neighboring sentence with mutation should be 
# putting 0 would mean genes would be considered only from sentence with mutation info
thres_neighboring_sent_count = 0


prev_line_with_mut = -9999999
paper_raw_info_compiled = dict()
# 'WBPaper ID', 'Method', 'Genes', 'WBGenes', 'Variants', '*Gene-Variant combo ', 'Mutations', 'Normalized Mutations', 'Sentence'
for row in data:
    ppr_id = row[0]
    norm_muts = row[-2]
    wbgenes = row[3]
    variants = row[4]
    line_number = int(row[-1].split()[1][:1])
    if ppr_id not in paper_raw_info_compiled.keys():
        paper_raw_info_compiled[ppr_id] = {'Mutations':[], 'WBGenes':[], 'Variants':[]}
        
    # filtering out nan values
    if type(norm_muts) != float:    
        prev_line_with_mut = line_number
        norm_muts = norm_muts[1:-1].split("', '")
        for m in norm_muts: paper_raw_info_compiled[ppr_id]['Mutations'].append(m)
    elif not (line_number - prev_line_with_mut) <= thres_neighboring_sent_count:
        continue
        
    if type(wbgenes) != float:
        wbgenes = wbgenes[1:-1].split("', '")
        for w in wbgenes: paper_raw_info_compiled[ppr_id]['WBGenes'].append(w)
    if type(variants) != float:
        variants = variants[1:-1].split("', '")
        for v in variants: paper_raw_info_compiled[ppr_id]['Variants'].append(v)

In [25]:
def unique_rows(a):
    a = np.ascontiguousarray(a)
    unique_a = np.unique(a.view([('', a.dtype)]*a.shape[1]))
    return unique_a.view(a.dtype).reshape((unique_a.shape[0], a.shape[1]))

In [26]:
matches = [] 
final_sheet = [] # ppr_id, gene, transcript

for ppr_id, info_from_ppr in paper_raw_info_compiled.items():
    wbgenes = info_from_ppr['WBGenes']
    mutations = info_from_ppr['Mutations']
    for gene in wbgenes:
        if gene not in wb_gene_and_prot.keys():
            continue
        for row in wb_gene_and_prot[gene]:
            transcript, protein_string = row
            for mut in mutations:
                if not len(mut):
                    continue
                wt_res = mut[0]
                pos = int(''.join(n for n in mut if n.isdigit()))
                mut_res = mut[-1]
                try:
                    if protein_string[pos-1] == wt_res:
                        matches.append([ppr_id, gene + ' ' + mut + ' ' + transcript])
                except IndexError:
                    pass
    
matches = unique_rows(matches)
for r in matches:
    p = r[0]
    g, m, t = r[1].split()
    final_sheet.append([p,g,m,t])

In [27]:
len(final_sheet)

1945

In [28]:
# saving things
data = pd.DataFrame(final_sheet[:], columns=['WBPaper ID', 'WBGenes', 'Mutations', 'Transcript'])
data.to_csv("data/model_output/processed/final.csv", index=False, encoding='utf-8')

# 5 Verification

In [29]:
data = pd.read_csv("data/model_output/processed/final.csv")
data = data.to_numpy() # ppr_id, gene, mutation, transcript
paper_ids_processed = np.unique(data[:,0])
paper_ids_processed = np.sort(paper_ids_processed)

temp = pd.read_csv("data/model_output/processed/snippets_1.csv")
temp = temp.to_numpy()
total_paper_ids_processed = np.unique(temp[:,0])
temp = None

In [30]:
print('Total count of papers processed and papers in the final processed sheet:', len(total_paper_ids_processed), '&', len(paper_ids_processed))

Total count of papers processed and papers in the final processed sheet: 100 & 68


In [31]:
ground_truth = Path('data/gsoc/Variants_best_outcome.txt').read_text().split('\n')
ground_truth = [r.split('\t') for r in ground_truth][:-1]
ground_truth = np.array(ground_truth, dtype=object)

In [32]:
# Checking if any processed paper is not in the ground truth file
for id in paper_ids_processed:
    if id not in ground_truth[:,0]:
        print(id)

WBPaper00030864


In [33]:
true_positives = []
false_positives = []
for row in data:
    paper_id = row[0]
    gene = row[1]
    mutation = row[2]
    # not sure if this makes sense in biology context but normalizing the stop codon names to X
    mutation = re.sub('(opal|umber|amber|ochre|stop|\*)', 'X', mutation, flags=re.I)
    transcript = row[3]
    bool_found = False
    for label in ground_truth[ground_truth[:,0] == paper_id]:
        label[-2] = re.sub('(opal|umber|amber|ochre|stop|\*)', 'X', label[-2], flags=re.I)
        if transcript == label[-1] and mutation == label[-2]:
            bool_found = True
            # continue bc we're storing all the labels from a paper
            continue
    if bool_found:
        true_positives.append(row)
    else:
        false_positives.append(row)

In [34]:
len(true_positives), len(false_positives)

(266, 1679)

In [35]:
print('Precision ',len(true_positives)*100/(len(true_positives) + len(false_positives)), '%')

Precision  13.676092544987146 %


### Checking how many matches are present in the ground truth for the processed papers

In [36]:
all_from_truth = []
for ppr in paper_ids_processed:
    for label in ground_truth[ground_truth[:,0] == ppr]:
        label[-2] = re.sub('(opal|umber|amber|ochre|stop|\*)', 'X', label[-2], flags=re.I)
        all_from_truth.append(label)

In [37]:
len(all_from_truth)

1140